[View in Colaboratory](https://colab.research.google.com/github/Rblivingstone/iMaterialist_Kaggle_competition/blob/master/notebook.ipynb)

In [1]:
!pip install kaggle
import keras
import pandas as pd


You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


Using TensorFlow backend.


In [2]:
from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth

auth.authenticate_user()

drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])

filename = "/content/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)
 



Download 100%.


In [3]:
!kaggle competitions download -c imaterialist-challenge-fashion-2018 -w

test.json.zip: Skipping, found more recently modified local copy (use --force to force download)
validation.json.zip: Skipping, found more recently modified local copy (use --force to force download)
train.json.zip: Skipping, found more recently modified local copy (use --force to force download)
sample_submission.csv.zip: Skipping, found more recently modified local copy (use --force to force download)


In [0]:
from shutil import unpack_archive
unpack_archive('train.json.zip')


In [3]:
import json
print('okay')
json_data=open('train.json').read()
print('cool')
data = json.loads(json_data)
print(data['images'])

okay


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [0]:
urls = [obj['url'] for obj in data['images']]
ids = [obj['imageId'] for obj in data['images']]

In [0]:
import urllib.request
try:
  os.mkdir('/images')
except FileExistsError:
  pass

In [0]:
for i in range(len(urls)):
  try:
     urllib.request.urlretrieve(urls[i],'/images/{0}.jpg'.format(ids[i]))
  except:ata
    print('{0} Failed to Download, Skipping ID: {1}'.format(urls[i],ids[i]))

https://contestimg.wish.com/api/webimage/5494fceff88ca01004df3b80-large Failed to Download, Skipping ID: 35
https://contestimg.wish.com/api/webimage/540040b8ecdd3054b095b0a1-large Failed to Download, Skipping ID: 55
https://contestimg.wish.com/api/webimage/544b91d564d8781016a15f18-large Failed to Download, Skipping ID: 120
https://contestimg.wish.com/api/webimage/544f6162180866102287a223-large Failed to Download, Skipping ID: 126
https://contestimg.wish.com/api/webimage/541d15b0037e1c1cc618e505-large Failed to Download, Skipping ID: 185
https://contestimg.wish.com/api/webimage/5455a6dbde557b10275cffb1-large Failed to Download, Skipping ID: 301
https://contestimg.wish.com/api/webimage/53fb1b260aae3009760c111d-large Failed to Download, Skipping ID: 344
https://contestimg.wish.com/api/webimage/540a9ce821f1610aa31aee3e-large Failed to Download, Skipping ID: 346
https://contestimg.wish.com/api/webimage/540bbf96814f8b0bb18a3f8c-large Failed to Download, Skipping ID: 349
https://contestimg.wi

https://contestimg.wish.com/api/webimage/5406f4a73cefbe7a7f06e05e-large Failed to Download, Skipping ID: 2080
https://contestimg.wish.com/api/webimage/54421f5887b47a6d833e46f7-large Failed to Download, Skipping ID: 2106
https://contestimg.wish.com/api/webimage/540ec38b0e2dba0a72922fb5-large Failed to Download, Skipping ID: 2114
https://contestimg.wish.com/api/webimage/54165d214974cf0972f75619-large Failed to Download, Skipping ID: 2129
https://contestimg.wish.com/api/webimage/548d114830d8b817879f7ef8-large Failed to Download, Skipping ID: 2140
https://contestimg.wish.com/api/webimage/54228ff8ac624b0a25e77904-large Failed to Download, Skipping ID: 2163
https://contestimg.wish.com/api/webimage/543b6e03b4bc390f8ac86699-large Failed to Download, Skipping ID: 2329
https://contestimg.wish.com/api/webimage/544b6d4cda76fa100bbd05c5-large Failed to Download, Skipping ID: 2362
https://contestimg.wish.com/api/webimage/541ff86d7ae53f0d04823a72-large Failed to Download, Skipping ID: 2374
https://co

In [11]:
import numpy as np
output = []
for dat in data['annotations']:
  temp = [0]*228
  for obj in dat['labelId']:
    temp[int(obj)-1] = 1
  output.append(temp)
depvar = np.array(output)
print(depvar)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [12]:
depvar[0,136]

1